# 네이버 웹툰 크롤링
https://yeeybook.tistory.com/139

https://github.com/yeeybook/WebCrawler/blob/master/naver_webtoon_crawler.py

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd

## (1) 연재 중 웹툰

In [2]:
URL = 'https://comic.naver.com/webtoon/weekday.nhn'
html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
soup = BeautifulSoup(html, 'html.parser')

In [3]:
title = soup.find_all('a', {'class' : 'title'}) # a태그에서 class='title'인 html소스를 찾아 할당
id_list = [] ; title_list = [] ; author_list = [] ; day_list = [] ; genre_list = [] ; story_list = [] ; platform_list = []
num = 0

driver = webdriver.Chrome('C:/chromedriver.exe') # 크롬 사용하니까
driver.get(URL)

for i in range(len(title)):
    sleep(0.5) # 크롤링 중간 중간 텀을 주어 과부하 생기지 않도록

    page = driver.find_elements_by_class_name('title')
    page[i].click() #월요일 첫 번째 웹툰부터 순서대로 클릭

    sleep(0.5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') # 이동한 페이지 주소 읽고 파싱

    day = soup.find_all('ul', {'class' : 'category_tab'})
    day = day[0].find('li', {'class' : 'on'}).text[0:1] # 요일 수집

    t = title[i].text
    if (t in title_list):  # 요일 두 개 이상이면 요일만 추가함
        day_list[title_list.index(t)] += ', ' + day
        driver.back()
        continue

    id_list.append(num) ; num += 1  # id 리스트에 추가
    title_list.append(t)  # 제목 리스트에 추가
    day_list.append(day) # 요일 리스트에 추가
    platform_list.append('네이버 웹툰') # 플랫폼 리스트에 추가

    author = soup.find_all('h2') # 두 번째 h2태그에 있음
    author = author[1].find('span', {'class' : 'wrt_nm'}).text[8:] # 7칸의 공백 후 8번 째부터 작가 이름임
    author_list.append(author) # 작가 리스트에 추가

    genre = soup.find('span', {'class' : 'genre'}).text # 장르 수집
    genre_list.append(genre) # 장르 리스트에 추가

    story = soup.find_all('p') # 줄거리 수집
    story = str(story[3])
    story = story.replace('<p>', '').replace('</p>', '').replace('<br/>', '\n') # <br>을 개행으로 바꾸기
    story_list.append(story) # 줄거리 리스트에 추가

    driver.back() # 뒤로 가기

<ipython-input-3-5a7962935e24>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/chromedriver.exe') # 크롬 사용하니까
<ipython-input-3-5a7962935e24>:11: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  page = driver.find_elements_by_class_name('title')


In [4]:
total_data1 = pd.DataFrame()
total_data1['id'] = id_list
total_data1['title'] = title_list
total_data1['author'] = author_list
total_data1['day'] = day_list
total_data1['genre'] = genre_list
total_data1['story'] = story_list
total_data1['platform'] = platform_list
total_data1.to_csv('네이버_웹툰.csv', encoding = 'utf-8-sig')

In [5]:
total_data1.head()

,id,title,author,day,genre,story,platform
0,0,참교육,채용택 / 한가람,월,"스토리, 액션",무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n&lt;부...,네이버 웹툰
1,1,신의 탑,SIU,월,"스토리, 판타지",자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년\n그리고 그런 소년을 시험하는 탑,네이버 웹툰
2,2,뷰티풀 군바리,설이 / 윤성원,월,"스토리, 드라마",'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!,네이버 웹툰
3,3,윈드브레이커,조용석,월,"스토리, 스포츠",혼자서 자전거를 즐겨타던 모범생 조자현.\n원치 않게 자전거 크루의 일에 자꾸 휘말...,네이버 웹툰
4,4,퀘스트지상주의,박태준 만화회사,월,"스토리, 드라마","[외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!\n공부, 싸...",네이버 웹툰


In [8]:
total_data1.shape

(547, 7)

## (2) 완결 웹툰

In [19]:
URL = 'https://comic.naver.com/webtoon/finish'
html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
soup = BeautifulSoup(html, 'html.parser')

In [7]:
mask = soup.find_all('a', {'class' : 'mask'}) # a태그에서 class='title'인 html소스를 찾아 할당
id_list = [] ; title_list = [] ; genre_list = [] ; story_list = [] 
num = 0

driver = webdriver.Chrome('C:/chromedriver.exe') # 크롬 사용하니까
driver.get(URL)

for i in range(len(mask)):
    sleep(0.5) # 크롤링 중간 중간 텀을 주어 과부하 생기지 않도록

    page = driver.find_elements_by_class_name('mask')
    page[i].click() #월요일 첫 번째 웹툰부터 순서대로 클릭

    sleep(0.5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') # 이동한 페이지 주소 읽고 파싱

    t = title[i].text

    id_list.append(num) ; num += 1  # id 리스트에 추가
    title_list.append(t)  # 제목 리스트에 추가

    genre = soup.find('span', {'class' : 'genre'}).text # 장르 수집
    genre_list.append(genre) # 장르 리스트에 추가

    story = soup.find_all('p') # 줄거리 수집
    story = str(story[3])
    story = story.replace('<p>', '').replace('</p>', '').replace('<br/>', '\n') # <br>을 개행으로 바꾸기
    story_list.append(story) # 줄거리 리스트에 추가

    driver.back() # 뒤로 가기

<ipython-input-7-43d170878254>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/chromedriver.exe') # 크롬 사용하니까


In [ ]:
total_data2 = pd.DataFrame()
total_data2['id'] = id_list
total_data2['title'] = title_list
total_data2['genre'] = genre_list
total_data2['story'] = story_list
total_data2.to_csv('네이버_웹툰_완결.csv', encoding = 'utf-8-sig')

In [ ]:
total_data2.head()

In [17]:
title = soup.find_all('a', {'class' : 'title'}) # a태그에서 class='title'인 html소스를 찾아 할당

In [10]:
title = soup.find_all('a', {'class' : 'title'}) # a태그에서 class='title'인 html소스를 찾아 할당
id_list = [] ; title_list = [] ; genre_list = [] ; story_list = [] 
num = 0

driver = webdriver.Chrome('C:/chromedriver.exe') # 크롬 사용하니까
driver.get(URL)

for i in range(len(title)):
    sleep(0.5) # 크롤링 중간 중간 텀을 주어 과부하 생기지 않도록

    page = driver.find_elements_by_class_name('title')
    page[i].click() #월요일 첫 번째 웹툰부터 순서대로 클릭

    sleep(0.5)

<ipython-input-10-002e5a7a2097>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/chromedriver.exe') # 크롬 사용하니까


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv
# 브라우저 생성
browser = webdriver.Chrome('C:/chromedriver.exe')

# 열기
browser.get('https://comic.naver.com/webtoon/finish')  # 네이버를 열어라
browser.implicitly_wait(10) # 로딩이 끝날 때까지 10초까지는 기다려줌



## 최종 데이터

In [ ]:
# id, title, genre, story 남기기
# 합치고 index 정리
# title 중복 제거
# 최종 데이터 저장